# Tweet Turing Test: Detecting Disinformation on Twitter  

|          | Group #2 - Disinformation Detectors                     |
|---------:|---------------------------------------------------------|
| Members  | John Johnson, Katy Matulay, Justin Minnion, Jared Rubin |
| Notebook | `xx_fine_tuner.ipynb`                                   |
| Purpose  | A notebook to fine-tune BERT models.                    |

*Assumptions*  
 - The dataset being used has binary class labels following convention: 0 = authentic tweet; 1 = troll tweet
 - The execution environment has internet access (to download models from huggingface.co)

*Notes*
 - Notebook is based on tutorial from: https://huggingface.co/docs/transformers/training
 - Additional help from: https://www.youtube.com/watch?v=V1-Hm2rNkik&t=351s

# 1 - Setup

In [1]:
# imports from Python standard library
import os
import json
import pathlib
import shutil
import zipfile

# imports requiring installation
#   connection to Google Cloud Storage
from google.cloud import storage            # pip install google-cloud-storage
from google.oauth2 import service_account   # pip install google-auth

#  data science packages
import numpy as np
import pandas as pd

# 🤗 (huggingface) packages
import evaluate
from datasets import Dataset, ClassLabel
from transformers import BertTokenizerFast, BertweetTokenizer, DistilBertTokenizerFast, RobertaTokenizerFast, XLMRobertaTokenizerFast
from transformers import BertForSequenceClassification, RobertaForSequenceClassification, AutoModelForSequenceClassification, AutoTokenizer
from transformers import TrainingArguments, Trainer

In [2]:
# imports from tweet_turing.py
import tweet_turing as tur      # note - different import approach from prior notebooks

# imports from tweet_turing_paths.py
from tweet_turing_paths import local_data_paths, local_snapshot_paths, gcp_data_paths, \
    gcp_snapshot_paths, gcp_project_name, gcp_bucket_name, gcp_key_file

In [3]:
# pandas options
pd.set_option('display.max_colwidth', None)

## Local or Cloud?

Decide here whether to run notebook with local data or GCP bucket data
 - if the working directory of this notebook has a "../data/" folder with data loaded (e.g. working on local computer or have data files loaded to a cloud VM) then use the "local files" option and comment out the "gcp bucket files" option
 - if this notebook is being run from a GCP VM (preferrably in the `us-central1` location) then use the "gcp bucket files" option and comment out the "local files" option

In [4]:
# option: local files
local_or_cloud: str = "local"   # comment/uncomment this line or next

# option: gcp bucket files
#local_or_cloud: str = "cloud"   # comment/uncomment this line or previous

# don't comment/uncomment for remainder of cell
if (local_or_cloud == "local"):
    data_paths = local_data_paths
    snapshot_paths = local_snapshot_paths
elif (local_or_cloud == "cloud"):
    data_paths = gcp_data_paths
    snapshot_paths = gcp_snapshot_paths
else:
    raise ValueError("Variable 'local_or_cloud' can only take on one of two values, 'local' or 'cloud'.")
    # subsequent cells will not do this final "else" check

In [5]:
# this cell only needs to run its code if local_or_cloud=="cloud"
#   (though it is harmless if run when local_or_cloud=="local")
gcp_storage_client: storage.Client = None
gcp_bucket: storage.Bucket = None

if (local_or_cloud == "cloud"):
    gcp_storage_client = tur.get_gcp_storage_client(project_name=gcp_project_name, key_file=gcp_key_file)
    gcp_bucket = tur.get_gcp_bucket(storage_client=gcp_storage_client, bucket_name=gcp_bucket_name)

# 2 - Load Dataset

Starting with the ten-percent sample with NLP-preprocessing completed from notebook **`04_nlp_preprocess.ipynb`**.

In [6]:
# note this cell requires package `pyarrow` to be installed in environment
parq_filename: str = "data_sample_ten_percent_NLP_preprocessed.parquet.gz"
parq_path: str = f"{snapshot_paths['parq_snapshot']}{parq_filename}"

if (local_or_cloud == "local"):
    df = pd.read_parquet(parq_path, engine='pyarrow')
elif (local_or_cloud == "cloud"):
    df = tur.get_gcp_object_from_parq_as_df(bucket=gcp_bucket, object_name=parq_path)

In [7]:
df.head(3)

,external_author_id,author,content,region,language,following,followers,updates,post_type,is_retweet,...,has_url,emoji_text,emoji_count,publish_date,class,following_ratio,class_numeric,RUS_lett_count,content_demoji,content_no_emoji
0,23785050,radiowoody,"To live dangerously on Friday the 13th, we're doing the radio show from the UNLUCKIEST place on earth! The @TennesseeTitans Locker Room!",Nashville Tennessee,en,2585,5710,2,NaN,0.0,...,0,[],0,2013-12-13 10:03:43+00:00,Verified,0.452635,0,0,"To live dangerously on Friday the 13th, we're doing the radio show from the UNLUCKIEST place on earth! The @TennesseeTitans Locker Room!","To live dangerously on Friday the 13th, we're doing the radio show from the UNLUCKIEST place on earth! The @TennesseeTitans Locker Room!"
1,59020162,matthewpouliot,@legsanity I like it. Almost like a free Gio. Pujols is still about as good of a bet as Gonzalez the rest of the way.,Florida,en,999,12637,0,replied_to,0.0,...,0,[],0,2015-04-26 20:13:58+00:00,Verified,0.079047,0,0,@legsanity I like it. Almost like a free Gio. Pujols is still about as good of a bet as Gonzalez the rest of the way.,@legsanity I like it. Almost like a free Gio. Pujols is still about as good of a bet as Gonzalez the rest of the way.
2,1656024374,IMISSOBAMA,Man servants can have a good purpose as long as they come with cash and don't touch me ever.,United States,en,473,760,4122,RETWEET,1.0,...,0,[],0,2016-12-24 13:12:00+00:00,Troll,0.621551,1,0,Man servants can have a good purpose as long as they come with cash and don't touch me ever.,Man servants can have a good purpose as long as they come with cash and don't touch me ever.


In [8]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362314 entries, 0 to 362313
Data columns (total 24 columns):
 #   Column              Non-Null Count   Dtype              
---  ------              --------------   -----              
 0   external_author_id  362314 non-null  string             
 1   author              362314 non-null  string             
 2   content             362314 non-null  string             
 3   region              344249 non-null  string             
 4   language            362314 non-null  category           
 5   following           362314 non-null  uint64             
 6   followers           362314 non-null  uint64             
 7   updates             362314 non-null  uint64             
 8   post_type           154729 non-null  category           
 9   is_retweet          362314 non-null  float64            
 10  account_category    362314 non-null  category           
 11  tweet_id            362314 non-null  string             
 12  tco1_step1      

In [9]:
df['class'].unique()

['Verified', 'Troll']
Categories (2, object): ['Troll', 'Verified']

In [10]:
df.loc[df['emoji_count'] > 3, ['content', 'content_demoji', 'content_no_emoji', 'emoji_count']].sample(5, random_state=3)

,content,content_demoji,content_no_emoji,emoji_count
232301,RT DebAlwaystrump: Congress looking CORRUPT♨ 2 America😠thinks we are STUPID⤵ MUELLER is a DIRTY COP🚨 Russia was 4 … https://t.co/sVXXXBIjSh,RT DebAlwaystrump: Congress looking CORRUPT:hot springs: 2 America:angry face:thinks we are STUPID:right arrow curving down: MUELLER is a DIRTY COP:police car light: Russia was 4 … https://t.co/sVXXXBIjSh,RT DebAlwaystrump: Congress looking CORRUPT 2 Americathinks we are STUPID MUELLER is a DIRTY COP Russia was 4 … https://t.co/sVXXXBIjSh,4
203651,RT Laura_K69: GOOD LUCK!! 🤣🤣🤣🤣 https://t.co/pc4FYSEscE,RT Laura_K69: GOOD LUCK!! :rolling on the floor laughing::rolling on the floor laughing::rolling on the floor laughing::rolling on the floor laughing: https://t.co/pc4FYSEscE,RT Laura_K69: GOOD LUCK!! https://t.co/pc4FYSEscE,4
32502,"TRIGGER WARNING: Beware before reading my new @Townhallcom column, ""Dating Tips For Prominent Democrats."" It yields maximum kurtness. 💥🔥💥🔥💥🔥💥🤷🏼‍♂️🔥💥🔥💥🔥💥 https://t.co/9EGBRNlw1j","TRIGGER WARNING: Beware before reading my new @Townhallcom column, ""Dating Tips For Prominent Democrats."" It yields maximum kurtness. :collision::fire::collision::fire::collision::fire::collision::man shrugging: medium-light skin tone::fire::collision::fire::collision::fire::collision: https://t.co/9EGBRNlw1j","TRIGGER WARNING: Beware before reading my new @Townhallcom column, ""Dating Tips For Prominent Democrats."" It yields maximum kurtness. https://t.co/9EGBRNlw1j",14
186365,RT @BNPPARIBASOPEN: How does @EVesnina001 lose her #BNPPO17 🏆 midway through her champion's press conference? 🎥😂➡️ https://t.co/RbDT7nlRt7,RT @BNPPARIBASOPEN: How does @EVesnina001 lose her #BNPPO17 :trophy: midway through her champion's press conference?\n\n:movie camera::face with tears of joy::right arrow: https://t.co/RbDT7nlRt7,RT @BNPPARIBASOPEN: How does @EVesnina001 lose her #BNPPO17 midway through her champion's press conference?\n\n https://t.co/RbDT7nlRt7,4
303189,RT @funsized411: 💯💯💯💯💯 RT @TheMisterMarcus: I'm okay with Duke losing even if it ruins my bracket.,RT @funsized411: :hundred points::hundred points::hundred points::hundred points::hundred points: RT @TheMisterMarcus: I'm okay with Duke losing even if it ruins my bracket.,RT @funsized411: RT @TheMisterMarcus: I'm okay with Duke losing even if it ruins my bracket.,5


# 3 - Choose Dataset Fields and Model

## 3.1 - Set Args

To make the subsequent encoding/training code more modular, set as many args as we can within this cell.

In [11]:
# Step 0 - Define model choices and their respective classes.
pretrained_models = {
    'bert-base-uncased': {
        'name': 'bert-base-uncased',          # https://huggingface.co/bert-base-uncased
        'tokenizer': BertTokenizerFast,
        'model': BertForSequenceClassification,
    },
    'distilbert-base-uncased': {                    # https://huggingface.co/distilbert-base-uncased
        'name': 'distilbert-base-uncased',
        'tokenizer': DistilBertTokenizerFast,
        'model': BertForSequenceClassification,
    },
    'roberta-base': {                               # https://huggingface.co/roberta-base
        'name': 'roberta-base',
        'tokenizer': RobertaTokenizerFast,          # note: roberta-base is case sensitive
        'model': BertForSequenceClassification,
    },
    'vinai/bertweet-base': {                        # https://huggingface.co/vinai/bertweet-base
        'name': 'vinai/bertweet-base',
        'tokenizer': BertweetTokenizer,             # note: bertweet-base is case sensitive
        'model': RobertaForSequenceClassification,  # note: this class is chosen by AutoModel
    },
    'Twitter/twhin-bert-base': {                    # https://huggingface.co/Twitter/twhin-bert-base
        'name': 'Twitter/twhin-bert-base',
        #'tokenizer': XLMRobertaTokenizerFast,       # twhin-bert's pre-training tokenizer is 'xlm-roberta-base' according to https://arxiv.org/pdf/2209.07562v1.pdf
        #'model': BertForSequenceClassification,
        'tokenizer': AutoTokenizer,
        'model': AutoModelForSequenceClassification,
    },
    # 'auto_vinai/bertweet-base': {                        # https://huggingface.co/vinai/bertweet-base
    #     'name': 'vinai/bertweet-base',
    #     'tokenizer': AutoTokenizer,
    #     'model': AutoModelForSequenceClassification,
    # },
}

In [12]:
# Step 1 - Select the pre-trained model (must be a key in `pretrained_models` above)
pretrained_model_choice = 'Twitter/twhin-bert-base'
pretrained_model = pretrained_models[pretrained_model_choice]

# Step 2 - Choose a descriptive name for this fine-tuning run.
#   Suggested format: [model name]-[2*max_tweets_per_class in thousands]k__[YYYY-MM-DD]
run_name = "Twitter__twhin-bert-base-10k__2023-02-22"

# Step 3 - Choose a folder name for where to store the output of the model 
#   Will be created as a subfolder of `../data/models/`
output_dir_name = 'Twitter__twhin-bert-base-10k'

# Step 4 - Choose which columns from dataframe will be used.
content_column = 'content_no_emoji'
class_column = 'class_numeric'          # Assumes: 0=authentic, 1=troll
pk_column = 'tweet_id'                  # Used to identify which tweets were used for fine tuning and exclude them from later testing

# Step 5A - Choose tokenizer args
#   Note: these are passed on to tokenizer object as keyword args
common_tokenizer_args = {
    'padding': 'max_length', 
    'truncation': True, 
    'return_tensors': 'pt', 
    'max_length': 256,
}

# Step 5B - Choose model args
#   Note: these are passed on to model object as keyword args
common_model_args = {
    'num_labels': 2,
    #'output_hidden_states': True,
}

# Step 5C - Choose trainer args
#   Note: these are passed on to trainer object as keyword args
#   Docs: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments
common_train_args = {
    #### model output
    'run_name': run_name,
    'output_dir': f'../data/models/{output_dir_name}',
    'save_strategy': 'epoch',
    'save_total_limit': 1,
    #### training hyperparams
    'num_train_epochs': 10,
    # 'per_device_train_batch_size': 16,
    # 'per_device_eval_batch_size': 16,
    # 'warmup_steps': 500,
    # 'weight_decay': 0.01,
    #### evaluation during training
    # 'evaluation_strategy': 'steps',
    # 'eval_steps': 100,
    'evaluation_strategy': 'epoch',
    'logging_strategy': 'epoch',
    'log_level': 'warning',
}

# Step 6 - Choose how many tweets to fine-tune with, and test split fraction size
# Notes:
#   - Sets maximum tweets (per class) used for fine tuning but not including evaluation (set to None for no limit)
#       e.g. If this value is 5000, a maximum of 5000 troll and 5000 authentic tweets will be used
#       for a total of 10,000 tweets used for fine tuning. If `test_fraction` is 0.2, then 2,000 additional
#       tweets will be used for testing the fine tuned model, so 12,000 total tweets will be ingested.
#   - `max_tweets_per_class` is assumed to be less than total number of tweets per class in `df` (or else pandas yells)
max_tweets_per_class = 5000
test_fraction = 0.20            # within range (0.0, 1.0)

# Step 7 - (Optional) Set random seed values for reproducability
sampling_random_seed = 42       # for reproducability, and "the answer to life, the universe, and everything"
train_test_random_seed = 3      # for reproducability, and "the number of the counting shall be three"


## 3.2 - Convert Pandas Dataframe to 🤗 Dataset

In [13]:
# for model summary we can track how long it took to encode and train
time_encoding = None
time_training = None

# create a view (not a copy) of dataframe
if (max_tweets_per_class is None):
    df_view = df[[content_column, class_column, pk_column]]
else:
    n_tweets = int(max_tweets_per_class * (1.0 + test_fraction))     # "gross up" the number of tweets ingested (see section 3.1 above)

    df_view = pd.concat(
        [
            df.loc[df[class_column] == 1, [content_column, class_column, pk_column]].sample(n=n_tweets, random_state=sampling_random_seed),
            df.loc[df[class_column] == 0, [content_column, class_column, pk_column]].sample(n=n_tweets, random_state=sampling_random_seed)
        ], 
        ignore_index=True
    )

# convert to 🤗 Dataset object
dataset = Dataset.from_pandas(df_view) \
            .rename_columns({content_column: "text", class_column: "label"}) \
            .cast_column("label", ClassLabel(names=['authentic', 'troll']))

# check results
assert (dataset.features['label'].str2int('authentic') == 0) and (dataset.features['label'].str2int('troll') == 1), 'class labels mismatched'
dataset[0]

Casting the dataset:   0%|          | 0/12 [00:00<?, ?ba/s]

{'text': 'On #MuslimWomensDay, these women are empowering themselves and fighting back against Islamophobia. https://t.co/Y5NXaTHjZi',
 'label': 1,
 'tweet_id': '846549212082909184'}

## 3.3 - Train/Test Split

In [14]:
if (max_tweets_per_class is None):
    test_size = test_fraction
else:
    test_size = int(max_tweets_per_class * test_fraction * 2)   # "2" for our two classes

dataset_split = dataset.train_test_split(
    test_size=test_size,
    shuffle=True,
    seed=train_test_random_seed,
    stratify_by_column='label'
)

# check output
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tweet_id'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label', 'tweet_id'],
        num_rows: 2000
    })
})

## 3.4 - Tokenize / Encode

In [15]:
# create the tokenizer to prepare text for model
#tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)
tokenizer = pretrained_model['tokenizer'].from_pretrained(pretrained_model['name'])

In [16]:
# create a tokenizer function
def tokenize_function(examples):
    return tokenizer(examples['text'], **common_tokenizer_args)

# todo -> convert to pure function
#   Use (next cell): tokenized_datasets = dataset_split.map(tokenize_function, batched=True, fn_kwargs={'tokenizer': tokenizer})

In [17]:
time_encoding_start = pd.Timestamp.now()

# encode the training and test sets
#tokenized_datasets = dataset_split.map(tokenize_function, batched=True, fn_kwargs={'tokenizer': tokenizer})
tokenized_datasets = dataset_split.map(tokenize_function, batched=True)

time_encoding_stop = pd.Timestamp.now()
time_encoding = time_encoding_stop - time_encoding_start

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [18]:
print("Encoding duration:", str(time_encoding), end="\n\n")
print(tokenized_datasets)

Encoding duration: 0 days 00:00:03.593261

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'tweet_id', 'input_ids', 'attention_mask'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['text', 'label', 'tweet_id', 'input_ids', 'attention_mask'],
        num_rows: 2000
    })
})


## 3.5 - Model

In [19]:
model = pretrained_model['model'].from_pretrained(
    pretrained_model['name'],
    **common_model_args,
)

Some weights of the model checkpoint at Twitter/twhin-bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-ba

In [20]:
# setup the training arguments
training_args = TrainingArguments(
    **common_train_args
)

In [21]:
# setup training / evaluation metric
#   Docs: https://huggingface.co/docs/evaluate/package_reference/main_classes#evaluate.combine
#   Each of these metrics corresponds to a script from huggingface, below are the links for each script.
#       accuracy:       https://huggingface.co/spaces/evaluate-metric/accuracy
#       f1:             https://huggingface.co/spaces/evaluate-metric/f1
#       precision:      https://huggingface.co/spaces/evaluate-metric/precision
#       recall:         https://huggingface.co/spaces/evaluate-metric/recall
#       roc_auc:        https://huggingface.co/spaces/evaluate-metric/roc_auc
#       brier_score:    https://huggingface.co/spaces/evaluate-metric/brier_score
metric_list = ['accuracy', 'f1', 'precision', 'recall', 'roc_auc', 'brier_score']

metric = evaluate.combine(evaluations=metric_list)

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels, prediction_scores=predictions)

In [22]:
time_training_start = pd.Timestamp.now()

# setup the trainer
#   Docs: https://huggingface.co/docs/transformers/main_classes/trainer#transformers.Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['test'],
    compute_metrics=compute_metrics,
)

# execute the training
trainer.train()

time_training_stop = pd.Timestamp.now()
time_training = time_training_stop - time_training_start

print("\nTraining duration:", str(time_training))

c:\Users\Justin\.envs\tf290_env\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/12500 [00:00<?, ?it/s]

{'loss': 0.4222, 'learning_rate': 4.5e-05, 'epoch': 1.0}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.35998088121414185, 'eval_accuracy': 0.826, 'eval_f1': 0.8042744656917885, 'eval_precision': 0.9190231362467867, 'eval_recall': 0.715, 'eval_roc_auc': 0.826, 'eval_brier_score': 0.174, 'eval_runtime': 18.8461, 'eval_samples_per_second': 106.123, 'eval_steps_per_second': 13.265, 'epoch': 1.0}
{'loss': 0.3368, 'learning_rate': 4e-05, 'epoch': 2.0}


...

{'eval_loss': 0.7302846908569336, 'eval_accuracy': 0.862, 'eval_f1': 0.8575851393188852, 'eval_precision': 0.8859275053304904, 'eval_recall': 0.831, 'eval_roc_auc': 0.862, 'eval_brier_score': 0.138, 'eval_runtime': 19.5306, 'eval_samples_per_second': 102.403, 'eval_steps_per_second': 12.8, 'epoch': 8.0}
{'loss': 0.0298, 'learning_rate': 5e-06, 'epoch': 9.0}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.8355393409729004, 'eval_accuracy': 0.866, 'eval_f1': 0.863265306122449, 'eval_precision': 0.88125, 'eval_recall': 0.846, 'eval_roc_auc': 0.866, 'eval_brier_score': 0.134, 'eval_runtime': 19.5477, 'eval_samples_per_second': 102.314, 'eval_steps_per_second': 12.789, 'epoch': 9.0}
{'loss': 0.0199, 'learning_rate': 0.0, 'epoch': 10.0}


  0%|          | 0/250 [00:00<?, ?it/s]

{'eval_loss': 0.8467128872871399, 'eval_accuracy': 0.865, 'eval_f1': 0.86404833836858, 'eval_precision': 0.8701825557809331, 'eval_recall': 0.858, 'eval_roc_auc': 0.865, 'eval_brier_score': 0.135, 'eval_runtime': 19.4782, 'eval_samples_per_second': 102.679, 'eval_steps_per_second': 12.835, 'epoch': 10.0}
{'train_runtime': 3900.2754, 'train_samples_per_second': 25.639, 'train_steps_per_second': 3.205, 'train_loss': 0.1685830812072754, 'epoch': 10.0}

Training duration: 0 days 01:05:00.996086


In [32]:
print(type(model))
print(type(tokenizer))

<class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'>
<class 'transformers.models.xlm_roberta.tokenization_xlm_roberta_fast.XLMRobertaTokenizerFast'>


## 3.6 - Save fine-tuned model

In [23]:
trainer.save_model()    # defaults to self.args.output_dir

In [24]:
# optional - delete checkpoint directories
checkpoint_dirs = [
    f"{trainer.args.output_dir}/{directory}"
    for directory in os.listdir(trainer.args.output_dir)
        if (
            os.path.isdir(os.path.join(trainer.args.output_dir, directory))
            and
            directory.startswith('checkpoint')
        )
]

for checkpoint_dir in checkpoint_dirs:
    print(f"Attempting to delete '{checkpoint_dir}' ...", end='')
    shutil.rmtree(checkpoint_dir)
    print(f" success")

Attempting to delete '../data/models/Twitter__twhin-bert-base-10k/checkpoint-12500' ... success


## 3.7 - Evaluate fine-tuned model

In [25]:
# if evaluating immediately after fine-tuning
#   Note: for current step, not using separate dataset for eval/test
train_metrics = trainer.evaluate(eval_dataset=tokenized_datasets['train'], metric_key_prefix='final_train')
test_metrics = trainer.evaluate(eval_dataset=tokenized_datasets['test'], metric_key_prefix='final_test')

print("Training Metrics:\n", train_metrics)
print("\nTest/Eval Metrics:\n", test_metrics)

  0%|          | 0/1250 [00:00<?, ?it/s]

  0%|          | 0/250 [00:00<?, ?it/s]

Training Metrics:
 {'final_train_loss': 0.014721373096108437, 'final_train_accuracy': 0.9979, 'final_train_f1': 0.997899789978998, 'final_train_precision': 0.997999599919984, 'final_train_recall': 0.9978, 'final_train_roc_auc': 0.9979, 'final_train_brier_score': 0.0021, 'final_train_runtime': 92.1231, 'final_train_samples_per_second': 108.55, 'final_train_steps_per_second': 13.569, 'epoch': 10.0}

Test/Eval Metrics:
 {'final_test_loss': 0.8467128872871399, 'final_test_accuracy': 0.865, 'final_test_f1': 0.86404833836858, 'final_test_precision': 0.8701825557809331, 'final_test_recall': 0.858, 'final_test_roc_auc': 0.865, 'final_test_brier_score': 0.135, 'final_test_runtime': 18.4041, 'final_test_samples_per_second': 108.672, 'final_test_steps_per_second': 13.584, 'epoch': 10.0}


In [26]:
for log_item in trainer.state.log_history:
    training_loss = log_item.get('loss', None)
    if (training_loss is not None):
        print(f"Epoch: {log_item['epoch']:>5.1f}", end=" ")     # "Epoch:   1.0 "
        print(f"| Training loss: {training_loss:>7.4f}")        # "| Training loss:  0.1234"

Epoch:   1.0 | Training loss:  0.4222
Epoch:   2.0 | Training loss:  0.3368
Epoch:   3.0 | Training loss:  0.2612
Epoch:   4.0 | Training loss:  0.1799
Epoch:   5.0 | Training loss:  0.1647
Epoch:   6.0 | Training loss:  0.1314
Epoch:   7.0 | Training loss:  0.0875
Epoch:   8.0 | Training loss:  0.0524
Epoch:   9.0 | Training loss:  0.0298
Epoch:  10.0 | Training loss:  0.0199


## 3.8 - Output key data as JSON

### 3.8.1 - Model Predictions

In [27]:
# make predictions
predictions = trainer.predict(tokenized_datasets['test'])

# predictions output is a subclass of NamedTuple, so to save to JSON we convert to a dict first
#   Note: according to python docs, the leading underscore is to avoid name conflicts, not as the usual "discouraged from use" meaning
#   Source: https://docs.python.org/3.10/library/collections.html#collections.somenamedtuple._asdict
predictions_dict = predictions._asdict()

predictions_dict['run_name'] = run_name
predictions_dict['predictions'] = predictions_dict['predictions'].tolist()
predictions_dict['label_ids'] = predictions_dict['label_ids'].tolist()

# sort the keys
dict_order = ['run_name', 'metrics', 'predictions', 'label_ids']
predictions_dict = {key: predictions_dict[key] for key in dict_order}

# save predictions to `output_dir`
predictions_filename = trainer.args.output_dir + '/predictions.json'
with open(predictions_filename, mode='w', encoding='utf-8') as fp:
    json.dump(predictions_dict, fp, indent=4)

  0%|          | 0/250 [00:00<?, ?it/s]

### 3.8.2 - Fine Tuning Parameters

In [28]:
# record all the fine-tuning parameters in a dict to be saved as JSON later (not used in actual fine-tuning code)
fine_tune_params_dict = {
    'run_name': run_name,
    'output_dir_name': output_dir_name,
    'content_column': content_column,
    'class_column': class_column,
    'pk_column': pk_column,
    'pretrained_model': {
        'name': pretrained_model['name'],
        'tokenizer': pretrained_model['tokenizer'].__name__,
        'model': pretrained_model['model'].__name__,
    },
    'common_tokenizer_args': common_tokenizer_args,
    'common_model_args': common_model_args,
    'common_train_args': common_train_args,
    'max_tweets_per_class': max_tweets_per_class,
    'sampling_random_seed': sampling_random_seed,
    'train_test_random_seed': train_test_random_seed,
    'test_fraction': test_fraction,
    'encoding_duration': str(time_encoding),
    'training_duration': str(time_training),
}

# save fine tuning params to `output_dir`
fine_tune_params_filename = trainer.args.output_dir + '/fine_tune_params.json'
with open(fine_tune_params_filename, mode='w', encoding='utf-8') as fp:
    json.dump(fine_tune_params_dict, fp, indent=4)

### 3.8.3 - Model Metrics

In [29]:
model_metrics: list = trainer.state.log_history   # same info as what is output at the end of each training epoch

# save metrics to `output_dir`
model_metrics_filename = trainer.args.output_dir + '/model_metrics.json'
with open(model_metrics_filename, mode='w', encoding='utf-8') as fp:
    json.dump(model_metrics, fp, indent=4)

### 3.8.4 - Tweets used for fine tuning

We'll want to exclude these tweets from being used in downstream training/evaluation tasks. Each tweet has a unique `tweet_id` value, stored conceptually as a 64-bit integer but physically as a string. We noted down the `tweet_id` values in Section 3.2 so let's store them to a JSON file here.

In [30]:
tweet_id_list = df_view[pk_column].to_list()

fine_tune_tweet_ids_filename = trainer.args.output_dir + '/fine_tune_tweet_ids.json'
with open(fine_tune_tweet_ids_filename, mode='w', encoding='utf-8') as fp:
    json.dump(tweet_id_list, fp, indent=4)

## 3.9 - Zip Model

Based on code from: https://realpython.com/python-zipfile/#creating-populating-and-extracting-your-own-zip-files

In [31]:
model_directory = "../data/models/"
zip_filename = pathlib.Path(model_directory, f"{run_name}.zip")

dir_to_zip = pathlib.Path(trainer.args.output_dir)

with zipfile.ZipFile(zip_filename, mode="w", compression=zipfile.ZIP_DEFLATED, compresslevel=6) as archive:
    for file_path in dir_to_zip.rglob("*"):
        archive.write(file_path, arcname=file_path.relative_to(dir_to_zip))